In [10]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
from datetime import datetime, date
import pandas as pd
from xtquant import xtdatacenter as xtdc
from xtquant import xtdata

xtdc.set_token('4065054877ce5724155dbc5bcba200381ce5eb35')
xtdc.init()

xtquant250516.1.1已经发布,前往 http://dict.thinktrader.net/nativeApi/download_xtquant.html 查看更新说明



In [13]:
xtdata.get_option_detail_data('TA2509C4700.ZF')

{'ExchangeID': 'CZCE',
 'InstrumentID': 'TA509C4700',
 'InstrumentName': 'TA509C4700',
 'ProductID': 'TAC',
 'ProductType': 2,
 'OpenDate': '20241113',
 'CreateDate': '20241113',
 'ExpireDate': '20250813',
 'PreClose': 165.0,
 'SettlementPrice': 160.5,
 'UpStopPrice': 494.5,
 'DownStopPrice': 0.5,
 'LongMarginRatio': 0.0,
 'ShortMarginRatio': 0.0,
 'PriceTick': 0.5,
 'VolumeMultiple': 5,
 'MaxMarketOrderVolume': 2,
 'MinMarketOrderVolume': 1,
 'MaxLimitOrderVolume': 100,
 'MinLimitOrderVolume': 1,
 'OptUnit': 5.0,
 'MarginUnit': 1.7976931348623157e+308,
 'OptUndlCode': 'TA2509',
 'OptUndlMarket': 'ZF',
 'OptUndlCodeFull': 'TA2509.ZF',
 'OptExercisePrice': 4700.0,
 'NeeqExeType': 2147483647,
 'OptUndlRiskFreeRate': 0.016462,
 'OptUndlHistoryRate': 0.2645948073603659,
 'EndDelivDate': '20250813',
 'optType': 'CALL',
 'ProductCode': 'TA.ZF'}

In [2]:
def cal_trading_days(start_date, end_date):
    """
    计算两个日期之间的交易日数量
    """
    trading_calendar = pd.read_pickle('trading_calendar.pkl')
    if isinstance(start_date, str):
        start_date = start_date
    elif isinstance(start_date, (datetime, date, pd.Timestamp)):
        start_date = pd.to_datetime(start_date).strftime('%Y%m%d')

    if isinstance(end_date, str):
        end_date = end_date
    elif isinstance(end_date, (datetime, date, pd.Timestamp)):
        end_date = pd.to_datetime(end_date).strftime('%Y%m%d')
    trading_days = trading_calendar[(trading_calendar[0] >= start_date) & (trading_calendar[0] <= end_date)]
    return len(trading_days)

In [24]:
def calc_natural_days(start_date, end_date):
    
    # 将字符串转换为datetime对象
    start_dt = datetime.strptime(start_date, '%Y%m%d')
    end_dt = datetime.strptime(end_date, '%Y%m%d')
    
    # 计算日期差值
    days_diff = (end_dt - start_dt).days
    
    return days_diff

In [3]:
def black76_price(F, K, T, r, sigma, option_type='call'):
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == 'put':
        return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")

In [4]:
def black76_iv(F, K, T, r, market_price, option_type='call'):
    """使用Brent法计算隐含波动率（严格校验option_type）"""
    # 第一步：校验option_type
    if option_type not in ['call', 'put']:
        raise ValueError("option_type必须是'call'或'put'")
    
    # 定义方程：模型价格 - 市场价格 = 0
    def equation(sigma):
        return black76_price(F, K, T, r, sigma, option_type) - market_price
    
    # 检查市场价格的合理性
    intrinsic_value = max(F - K, 0) if option_type == 'call' else max(K - F, 0)
    if market_price < intrinsic_value:
        raise ValueError(f"{option_type}价格低于内在价值（无套利条件不成立）")
    
    # 设置波动率搜索范围（0.1%到500%）
    sigma_min, sigma_max = 0.001, 5.0
    
    # 调用Brent法求解
    try:
        iv = brentq(equation, a=sigma_min, b=sigma_max, rtol=1e-6)
        return iv
    except ValueError:
        raise ValueError(f"未找到解，请检查输入参数或扩大搜索区间")

In [53]:
start_date = '20250630'
end_date = '20250813'
F = 1724
K = 1780
r = 0.02
market_price = 38.5

In [54]:
black76_iv(F=F, K=K, T = calc_natural_days(start_date, end_date) / 365, r=r, market_price=market_price, option_type='call')

0.2582789652940749

In [55]:
calc_implied_volatility(market_price = market_price, S = F, K = K, T = calc_natural_days(start_date, end_date) / 365, r = r, option_type='call')

0.2513386820438787

In [56]:
black76_iv(F=F, K=K, T = cal_trading_days(start_date, end_date) / 252, r=r, market_price=market_price, option_type='call')

0.24784011862356428

In [57]:
calc_implied_volatility(market_price = market_price, S = F, K = K, T = cal_trading_days(start_date, end_date) / 252, r = r, option_type='call')

0.24060259657395086

In [ ]:
# 当前参数
F0 = 100     # 当前期货价格
K = 105      # 行权价
T = 30/365   # 30天到期
r = 0.05     # 无风险利率
sigma = 0.2  # 波动率

# 当前看涨期权价格
call_price = black76(F0, K, T, r, sigma, 'call')
print(f"当前看涨期权价格: {call_price:.2f}")

# 压力测试：期货价格上涨10%
F_new = F0 * 1.10
call_price_new = black76(F_new, K, T, r, sigma, 'call')
pnl = call_price_new - call_price
print(f"上涨10%后期权价格: {call_price_new:.2f}")
print(f"预期损益（PnL）: {pnl:.2f}")

当前看涨期权价格: 0.64
上涨10%后期权价格: 5.70
预期损益（PnL）: 5.06


In [16]:
def BS_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return call_price

def BS_put(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

    return put_price

def calc_implied_volatility(market_price, S, K, T, r, option_type):
    if option_type == 'call':
        pricing_func = BS_call
        instrinsic_value = max(S - K, 0)
    else:
        pricing_func = BS_put
        instrinsic_value = max(K - S, 0)

    if market_price <= instrinsic_value:
        print(f"市场价格 {market_price} 小于或等于内在价值 {instrinsic_value}，无法计算隐含波动率")
        return None
    
    def objective_function(sigma):
        try:
            theoretical_price = pricing_func(S, K, T, r, sigma)
            return theoretical_price - market_price
        except Exception as e:
            print(f"计算理论价格时出错: {e}")
            return float('inf')
        
    try:
        sigma = brentq(objective_function, 0.001, 5.0,xtol = 1e-6,  maxiter=100)
        return sigma
    except Exception as e:
        print(f"计算隐含波动率时出错: {e}")
        return None

In [54]:
calc_implied_volatility(115, 4782, 4800, 0.128767, 0.02, 'call')

0.17201096344375894

In [43]:
def BS_call(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return call_price

In [ ]:
def black76_price(F, K, T, r, sigma, option_type='call'):
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        return np.exp(-r * T) * (F * norm.cdf(d1) - K * norm.cdf(d2))
    elif option_type == 'put':
        return np.exp(-r * T) * (K * norm.cdf(-d2) - F * norm.cdf(-d1))
    else:
        raise ValueError("option_type must be 'call' or 'put'")